In [0]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/ECE232_Project4')
from google.colab import files

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install python-igraph

     |████████████████████████████████| 3.2MB 2.8MB/s 


In [0]:
import csv
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import igraph as ig
from scipy.spatial import Delaunay

In [0]:
graph_info = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ECE232_Project4/los_angeles-censustracts-2019-4-All-MonthlyAggregate.csv')
graph_info = graph_info.drop(['standard_deviation_travel_time', 'geometric_mean_travel_time', 'geometric_standard_deviation_travel_time'], axis=1)
graph_info = graph_info.drop(graph_info[graph_info.month != 12].index)
dec_only = graph_info.drop(['month'], axis=1)
dec_info = np.asarray(dec_only) 
paths = {}
locations = np.unique(np.concatenate([dec_info[:,0],dec_info[:,1]]))
counter = 0
for row in dec_info:
    location = tuple(np.sort(row[0:2]))
    if location in paths:
        paths[location].append(row[2])
        counter += 1
    else:
        paths[location] = [row[2]]
with open('/content/drive/My Drive/Colab Notebooks/ECE232_Project4/december_only.txt','w') as f:
    for locations in paths:
        string = '{} {} {:.2f}\n'.format(int(locations[0]),int(locations[1]),np.mean(paths[locations]))
        f.write(string)

# Question 6

In [0]:
G = ig.Graph.Read(f = '/content/drive/My Drive/Colab Notebooks/ECE232_Project4/december_only.txt', format = 'ncol', directed = False)
gcc = G.components().giant()
ig.summary(G)
ig.summary(gcc)

IGRAPH UNW- 2649 1004955 -- 
+ attr: name (v), weight (e)
IGRAPH UNW- 2649 1004955 -- 
+ attr: name (v), weight (e)


# Question 7

In [0]:
with open('/content/drive/My Drive/Colab Notebooks/ECE232_Project4/los_angeles_censustracts.json') as json_data:
  geo_info = json.load(json_data)

geo_info = geo_info['features']

my_dict = {}

for info in geo_info:
    entry = {}
    coord = info['geometry']['coordinates'][0][0]
    if len(coord) > 2:
      mean_coord = np.mean(np.asarray(coord),axis=0)
    else:
      mean_coord = coord
    movement_id = info['properties']['MOVEMENT_ID']
    entry['coord'] = coord
    entry['mean_coord'] = mean_coord
    entry['display_name'] = info['properties']['DISPLAY_NAME']
    my_dict[movement_id] = entry
print(my_dict)

{'1': {'coord': [-118.11683, 34.107225], 'mean_coord': [-118.11683, 34.107225], 'display_name': 'Census Tract 480302'}, '2': {'coord': [-118.134669, 34.098771], 'mean_coord': [-118.134669, 34.098771], 'display_name': 'Census Tract 480303'}, '3': {'coord': [-118.134532, 34.091566], 'mean_coord': [-118.134532, 34.091566], 'display_name': 'Census Tract 480304'}, '4': {'coord': [-118.12276, 34.10447], 'mean_coord': [-118.12276, 34.10447], 'display_name': 'Census Tract 480400'}, '5': {'coord': [-118.141784, 34.098765], 'mean_coord': [-118.141784, 34.098765], 'display_name': 'Census Tract 480500'}, '6': {'coord': [-118.152124, 34.098665], 'mean_coord': [-118.152124, 34.098665], 'display_name': 'Census Tract 480704'}, '7': {'coord': [-118.146397, 34.087877], 'mean_coord': [-118.146397, 34.087877], 'display_name': 'Census Tract 480802'}, '8': {'coord': [-118.154182, 34.095204], 'mean_coord': [-118.154182, 34.095204], 'display_name': 'Census Tract 480803'}, '9': {'coord': [-118.155292, 34.09857

In [0]:
mst = gcc.spanning_tree(weights = gcc.es["weight"])
vertices = mst.vs()
ig.summary(mst)

IGRAPH UNW- 2649 2648 -- 
+ attr: name (v), weight (e)


In [0]:
counter = 1
for e in mst.es():
    st = e.tuple
    print(my_dict[vertices[st[0]]['name']]['display_name'],'---',my_dict[vertices[st[1]]['name']]['display_name'])
    if counter == 10:
      break
    counter += 1

# https://data.lacounty.gov/widgets/rv2f-zsc7
# https://geohub.lacity.org/datasets/enriched-la-county-census-tracts-2015/
# 554001 -- 554002 Bellflower city
# 461700 -- 460800 Pasadena city
# 302201 -- 302202 Glendale city
# 407101 -- 407002 La Puente city
# 433401 -- 433402 El Monte city
# 543603 -- 294410 Carson city -- Los Angeles city
# 482001 -- 530700 Monterey Park city
# 460800 -- 463800 Pasadena city
# 269100 -- 217001 Los Angeles city
# 011000 -- 001901 ??? 

Census Tract 554001 --- Census Tract 554002
Census Tract 461700 --- Census Tract 460800
Census Tract 302201 --- Census Tract 302202
Census Tract 407101 --- Census Tract 407002
Census Tract 433401 --- Census Tract 433402
Census Tract 543603 --- Census Tract 294410
Census Tract 482001 --- Census Tract 530700
Census Tract 460800 --- Census Tract 463800
Census Tract 269100 --- Census Tract 217001
Census Tract 011000 --- Census Tract 001901


# Question 8

In [0]:
from igraph import *
import random

total = 1000
max_index = len(gcc.vs()) - 1  
edge = EdgeSeq(gcc)
used = set()
satisfied = 0
counter = 0

def check_triangle():
    current = ()
    indices = []
    weights = []
    for i in range(0,3):
        indices.append(random.randint(0,max_index))        
    current = tuple(sorted(indices))
    while(current in used):
        for i in range(0,3):
            indices.append(random.randint(0,max_index))        
        current = tuple(sorted(indices))  
    for i in current:
        weights.append(edge[i]["weight"])
    if weights[0] < weights[1] + weights[2] and weights[1] < weights[0] + weights[2] and weights[2] < weights[0] + weights[1]:
        return True, current
    else:
        return False, current

while counter < total:              
    for i in range(0,3):
        satisfy, current_set = check_triangle()
        if satisfy:
            break                  
    used.add(current_set)
    if satisfy:
        satisfied += 1
    else:
        print('triangle not satisfied')
    counter += 1  
    
print('percent:', str(satisfied/total))

triangle not satisfied
triangle not satisfied
triangle not satisfied
triangle not satisfied
triangle not satisfied
triangle not satisfied
triangle not satisfied
triangle not satisfied
triangle not satisfied
triangle not satisfied
triangle not satisfied
triangle not satisfied
triangle not satisfied
triangle not satisfied
triangle not satisfied
triangle not satisfied
triangle not satisfied
triangle not satisfied
percent: 0.982


# Question 9

In [0]:
def a_walk(mst, walk, visited, vset):
    current = walk[-1]
    if len(walk) > 1 and current == walk[0]: 
      return True
    else:
        edges = mst.es.select(_between = ([current],vset))
        for edge in edges:
            if not edge in visited:
                visited.add(edge)
                node_tuple = edge.tuple
                if node_tuple[0] == current:
                    next_node = node_tuple[1]
                else:
                    next_node = node_tuple[0]
                walk.append(next_node)
                next_walk = a_walk(mst, walk, visited, vset)
                if next_walk:
                    return True
                else:
                    walk.pop()
                    visited.remove(edge)       
        return False
            
def euler(mst, v, vset):
    edges = mst.es.select(_source_in = [v])
    if len(edges) == 0: 
      return [v] 
    else:
        walk = [v]
        visited = set()
        a_walk(mst, walk, visited, vset)
        for i in range(len(walk) - 1):
            walk_edges = mst.es.select(_between = ([walk[i]], [walk[i+1]]))
            mst.delete_edges(walk_edges[0])
        results = []
        for node in walk:
            next_walk = euler(mst, node, vset)
            for e in next_walk:
                results.append(e)   
        return results
            
def algorithm(g):
    my_mst = g.spanning_tree(weights = gcc.es["weight"])
    mst_edges = my_mst.es()
    edge_list = []
    for edge in mst_edges:
      edge_list.append(edge.tuple)
    mst_temp = mst.as_undirected()
    mst_temp.add_edges(edge_list)
    mst_double = mst_temp.as_undirected()
    e_walk = euler(mst_double, 0, mst_double.vs())
    path = []
    visited = set()
    for i in e_walk:
      if not i in visited:
          path.append(i)
          visited.add(i)  
    total_weight = 0
    final_path = []
    for i in range(len(path) - 1):
      i_1 = path[i]
      i_2 = path[i + 1]
      edges = g.es.select(_between = ([i_1], [i_2]))
      if len(edges) > 0:
          total_weight += edges[0]['weight']
          final_path.append(i_1)
      else:
          total_weight += gcc.shortest_paths([i_1], [i_2], weights = gcc.es()['weight'])[0][0]
    return mst, total_weight, final_path

In [0]:
import sys
sys.setrecursionlimit(10000)
my_mst, total_weight, final_path = algorithm(gcc)
print('upper bound:', total_weight / sum(mst.es['weight']))

upper bound: 1.7300968643282872


# Question 10

In [0]:
latitude = []
longitude = []

for i in final_path:
    latitude.append(my_dict[my_mst.vs[i]['name']]['mean_coord'][1])
    longitude.append(my_dict[my_mst.vs[i]['name']]['mean_coord'][0])

plot_lati = np.hstack(latitude)
plot_longi = np.hstack(longitude)

# Calculate Santa's total distance

In [0]:
def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))

total_distance = 0

for i in range(len(plot_lati)-1):
  lati1 = plot_lati[i]
  lati2 = plot_lati[i+1]
  longi1 = plot_longi[i]
  longi2 = plot_longi[i+1]
  temp_result = haversine(lati1,longi1,lati2,longi2)
  total_distance = total_distance + temp_result

total_distance = round(total_distance,3)
print(total_distance) # unit km

4364.616


# Calculate Santa's speed

In [0]:
# define time period to be 10 p.m. to 4 a.m.
# so total time is 6 hours
santa_speed = 4364.616/6.0
santa_speed = round(santa_speed,3)
print(santa_speed) # unit km/hr

727.436


In [0]:
plt.figure(figsize=(20,15))
plt.plot(plot_longi, plot_lati)
plt.title('Trajectory of Santa')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

# Question 11

In [0]:
Graph.write_ncol(gcc,'/content/drive/My Drive/Colab Notebooks/ECE232_Project4/uber_gcc.txt')
uber_gcc = np.loadtxt('/content/drive/My Drive/Colab Notebooks/ECE232_Project4/uber_gcc.txt')
uber_gcc_graph = {}
for row in uber_gcc:
    location_index = tuple(np.sort(row[0:2]).astype('int'))
    uber_gcc_graph[location_index] = row[2]
uber_gcc_ids = np.concatenate([uber_gcc[:,0],uber_gcc[:,1]])
uber_gcc_ids = np.unique(uber_gcc_ids).astype('int')

mean_coords = np.zeros((len(uber_gcc_ids),2))
for n,location_key in enumerate(uber_gcc_ids):
    mean_coords[n,:] = my_dict[str(location_key)]['mean_coord']
tri = Delaunay(mean_coords)

In [0]:
plt.triplot(mean_coords[:,0], mean_coords[:,1], tri.simplices.copy())
plt.plot(mean_coords[:,0], mean_coords[:,1],'r.')
plt.xlabel('Degrees Longitude')
plt.ylabel('Degrees Latitude')
plt.title('Triangulated Roads')
plt.show()

# Question 13

In [0]:
MILES_PER_DEGREE_LATLON = 69

def get_time(loc1,loc2,transit_time_graph):
    if (loc1,loc2) in transit_time_graph.keys():
        return transit_time_graph[(loc1,loc2)]
    else:
        return 0

# Calculate flow as #cars/hour
def get_flow(loc1, loc2, transit_time_graph,location_dict):
    coord1 = location_dict[str(loc1)]['mean_coord']
    coord2 = location_dict[str(loc2)]['mean_coord']
    distance_miles = MILES_PER_DEGREE_LATLON * np.sqrt(np.sum(np.square(np.stack(coord2)-np.stack(coord1))))
    if (loc1,loc2) in transit_time_graph.keys():
        travel_time_hours = transit_time_graph[(loc1,loc2)] / 3600
        speed_mph = distance_miles / travel_time_hours
        flow = (2*speed_mph) / (0.003 + (speed_mph/1800))
        return flow
    else:
        return 0


def add_triangulated_edge(graph, triangle):
    edge12 = tuple(np.sort(triangle[:2]))
    edge13 = tuple(np.sort(triangle[0::2]))
    edge23 = tuple(np.sort(triangle[1:]))   

    flow12 = get_flow(edge12[0],edge12[1],uber_gcc_graph,my_dict)
    flow13 = get_flow(edge13[0],edge13[1],uber_gcc_graph,my_dict)
    flow23 = get_flow(edge23[0],edge23[1],uber_gcc_graph,my_dict)
    if edge12 not in graph.keys() and flow12 > 0:
        graph[edge12] = flow12
    if edge13 not in graph.keys() and flow13 > 0:
        graph[edge13] = flow13
    if edge23 not in graph.keys() and flow23 > 0:
        graph[edge23] = flow23

def add_travel_time_edge(graph,triangle,thresh):
    edge12 = tuple(np.sort(triangle[:2]))
    edge13 = tuple(np.sort(triangle[0::2]))
    edge23 = tuple(np.sort(triangle[1:]))   
    
    time12 = get_time(edge12[0],edge12[1],uber_gcc_graph)
    time13 = get_time(edge13[0],edge13[1],uber_gcc_graph)
    time23 = get_time(edge23[0],edge23[1],uber_gcc_graph)
    
    if edge12 not in graph.keys() and time12 < thresh and time12 > 0:
        graph[edge12] = time12
    if edge13 not in graph.keys() and time13 < thresh and time13 > 0:
        graph[edge13] = time13
    if edge23 not in graph.keys() and time23 < thresh and time23 > 0:
        graph[edge23] = time23

In [0]:
triangulated_graph = {}

for triangle_idx in tri.simplices:
    triangle = [uber_gcc_ids[triangle_idx[0]], uber_gcc_ids[triangle_idx[1]], uber_gcc_ids[triangle_idx[2]]]
    add_triangulated_edge(triangulated_graph,triangle)

In [0]:
with open('/content/drive/My Drive/Colab Notebooks/ECE232_Project4/triangulated_graph.txt','w') as f:
    for locations in triangulated_graph:
        string = '{} {} {:.2f}\n'.format(int(locations[0]),int(locations[1]),triangulated_graph[locations])
        f.write(string)

In [0]:
diff = np.ones(2716)
for i in my_dict.keys():
  diff[int(i)-1] = (np.stack(my_dict[i]['mean_coord'])[0]-(-118.56))**2+(np.stack(my_dict[i]['mean_coord'])[1]-(34.04))**2
print(np.argmin(diff)) 
for i in my_dict.keys():
  diff[int(i)-1] = (np.stack(my_dict[i]['mean_coord'])[0]-(-118.18))**2+(np.stack(my_dict[i]['mean_coord'])[1]-(33.77))**2
print(np.argmin(diff))

1522
665


In [0]:
Malibu_coords = my_dict['1522']['mean_coord']
plt.triplot(mean_coords[:,0], mean_coords[:,1], tri.simplices.copy())
plt.plot(mean_coords[:,0], mean_coords[:,1],'r.')
plt.plot(Malibu_coords[0],Malibu_coords[1],'k.',ms=20)
plt.xlim((Malibu_coords[0]-0.1,Malibu_coords[0]+0.1))
plt.ylim((Malibu_coords[1]-0.1,Malibu_coords[1]+0.1))
plt.xlabel('Degrees Longitude')
plt.ylabel('Degrees Latitude')
plt.title('Triangulated Roads Near Malibu (black dot)')
plt.show()

In [0]:
LongBeach_coords = my_dict['665']['mean_coord']
plt.triplot(mean_coords[:,0], mean_coords[:,1], tri.simplices.copy())
plt.plot(mean_coords[:,0], mean_coords[:,1],'r.')
plt.plot(LongBeach_coords[0],LongBeach_coords[1],'k.',ms=20)
plt.xlim((LongBeach_coords[0]-0.1,LongBeach_coords[0]+0.1))
plt.ylim((LongBeach_coords[1]-0.1,LongBeach_coords[1]+0.1))
plt.xlabel('Degrees Longitude')
plt.ylabel('Degrees Latitude')
plt.title('Triangulated Roads Near Long Beach (black dot)')
plt.show()

# Question 14

In [0]:
triangulated_time_graph = {}
THRESH = 60*12

for triangle_idx in tri.simplices:
    triangle = [uber_gcc_ids[triangle_idx[0]], uber_gcc_ids[triangle_idx[1]], uber_gcc_ids[triangle_idx[2]]]
    add_travel_time_edge(triangulated_time_graph, triangle, THRESH)
    
len(triangulated_time_graph)

In [0]:
plt.figure()
counter = 0
for edges in triangulated_time_graph.keys():
    coords1 = my_dict[str(edges[0])]['mean_coord']
    coords2 = my_dict[str(edges[1])]['mean_coord']
    plt.plot([coords1[0],coords2[0]],[coords1[1],coords2[1]],color='c')
plt.plot(mean_coords[:,0], mean_coords[:,1],'r.')

# Question 15

In [0]:
Malibu_coords = my_dict['1522']['mean_coord']
for edges in triangulated_time_graph.keys():
    coords1 = my_dict[str(edges[0])]['mean_coord']
    coords2 = my_dict[str(edges[1])]['mean_coord']
    plt.plot([coords1[0],coords2[0]],[coords1[1],coords2[1]],color='b')
plt.plot(Malibu_coords[0],Malibu_coords[1],'k.',ms=20)
plt.xlim((Malibu_coords[0]-0.1,Malibu_coords[0]+0.1))
plt.ylim((Malibu_coords[1]-0.1,Malibu_coords[1]+0.1))
plt.xlabel('Degrees Longitude')
plt.ylabel('Degrees Latitude')
plt.title('Pruned Path At Malibu')
plt.show()

In [0]:
LongBeach_coords = my_dict['665']['mean_coord']
for edges in triangulated_time_graph.keys():
    coords1 = my_dict[str(edges[0])]['mean_coord']
    coords2 = my_dict[str(edges[1])]['mean_coord']
    plt.plot([coords1[0],coords2[0]],[coords1[1],coords2[1]],color='b')
plt.plot(LongBeach_coords[0],LongBeach_coords[1],'k.',ms=20)
plt.xlim((LongBeach_coords[0]-0.1,LongBeach_coords[0]+0.1))
plt.ylim((LongBeach_coords[1]-0.1,LongBeach_coords[1]+0.1))
plt.xlabel('Degrees Longitude')
plt.ylabel('Degrees Latitude')
plt.title('Pruned Path At Long Beach')
plt.show()